AI Semester Project

In [66]:
import numpy as np
import math
import random
import string
import re
import time
import pandas as pd
import matplotlib.pyplot as plt

DATA GENERATION

In [67]:
def generate_sections(section_count,total_sems,isOdd):
  start=1
  if isOdd==True:
    start=0
  sections=set()
  for i in range(start,total_sems,2):
    for j in range(section_count):
      section_type=random.randint(0,2)
      strength=random.randint(30,60)
      if section_type==2:
        strength+=60
      sections.add(str(i+1)+','+str(chr(j+ord('A')))+','+str(strength))
  return sections

In [68]:
def assign_profs(courses,labs,sections,course_sections,n_profs,name_length=3):
  profs_names={}
  profs_ids={}
  availibility={}
  profs_assignment={}
  letters = string.ascii_lowercase
  name = ''.join(random.choice(letters) for i in range(name_length))
  for i in range(n_profs):
    while(name in profs_ids):
      name = ''.join(random.choice(letters) for i in range(name_length))
    name='Professor '+name
    profs_names[i]=name
    profs_ids[name]=i
    availibility[i]=3
  current_prof=0
  for i in course_sections:
    profs_assignment[i]=current_prof
    availibility[current_prof]-=1
    if availibility[current_prof]==0:
      current_prof+=1

  return profs_names,profs_ids,profs_assignment

In [69]:
def generate_course_sections(courses,labs,sections):
  course_sections=set()
  years=set()
  for i in sections:
    idx=int(i[0])-1
    if idx<len(courses):
      for j in range(len(courses[idx])):
        course_sections.add(i + "," + courses[idx][j])
    if idx<len(labs):
        course_sections.add(i + "," + labs[idx])

  return course_sections

In [70]:
def generate_classrooms(blocks,floors,classes):
  classrooms=set()
  for i in range(blocks):
    block=str(chr(i+ord('A')))
    for j in range(floors):
      size=60
      for k in range(classes):
        if(k>classes/3+1 or k==classes-1):
          size=120
        classrooms.add(block + "," + str(j+1) + str(k+1) + "," + str(size))
  return classrooms

In [71]:
def index_map(arr):
  index_map={}
  reverse_map={}
  current_code=0
  for i in arr:
    index_map[i]=current_code
    reverse_map[current_code]=i
    current_code+=1
  return index_map,reverse_map

In [72]:
def write_to_file(solution,classrooms,course_sections,profs):
    days={
        1:'Monday',
        2:'Tuesday',
        3:'Wednesday',
        4:'Thursday',
        5:'Friday'
    }
    times={
        0:'8:30-9:50',
        1:'10:00-11:20',
        2:'11:30-12:50',
        3:'01:00-02:20',
        4:'02:30-03:50',
        5:'04:00-05:20'
    }
    class_maps=index_map(classrooms)
    cs_maps=index_map(course_sections)

    time_table=list()
    
    class_names=[]
    for i in class_maps[0].keys():
        temp=re.split(',',i)
        class_names.append(temp[0]+'-'+temp[1])
    for i in range(5):
        time_table.append(pd.DataFrame(index=class_names,columns=['8:30-9:50','10:00-11:20','11:30-12:50','01:00-02:20','02:30-03:50','04:00-05:20']))
    for i in range(len(solution)):
        classroom1=class_maps[1][solution[i][5]-1]
        classroom2=class_maps[1][solution[i][8]-1]
        course_section=cs_maps[1][i]
        professor=profs[solution[i][2]]
        day1=solution[i][3]-1
        time1=solution[i][4]-1
        day2=solution[i][6]-1
        time2=solution[i][7]-1
        temp=re.split(',',classroom2)
        class1=temp[0]+'-'+temp[1]
        temp=re.split(',',classroom1)
        class2=temp[0]+'-'+temp[1]
        
        t1=course_section+' '+professor
        temp=re.split(',',t1)
        table_entry=temp[0]+temp[1]+' '+temp[3]
        if(len(temp)>4):
            table_entry+=' '+temp[4]
        time_table[day1][times[time1]][class1]=table_entry
        time_table[day2][times[time2]][class2]=table_entry

    for i in range(1,6):
        time_table[i-1].to_csv(days[i]+'.csv')  
    return

GENETIC ALGO

In [73]:
def binary_encode(chromosome,len_list):
#new_chromosome=np.empty(chromosome.shape,dtype=np.uint64)
    new_chromosome=[]
    for i in range(len(chromosome)):
        str_idx=''
        for j in range(len(chromosome[i])):
            length=int(math.ceil(math.log(len_list[j],2)))
            length_str='0'+str(length+1)+'b'
            str_idx+=(format(chromosome[i][j],length_str))
        new_chromosome.append([str_idx])
    return new_chromosome

In [74]:
def crossover(c1,c2):
  cpoint=random.randint(1,len(c1)-2)
  child1=np.vstack((c1[:cpoint],c2[cpoint:]))
  child2=np.vstack((c2[:cpoint],c1[cpoint:]))
  c1=child1
  c2=child2
  return

In [75]:
def mutate(c1,rooms):
  random_index=random.randint(0,len(c1)-1)
  total_days=5
  time_slots=6
  cs=random_index
  isLab=True if c1[cs][0]==1 else False
  if isLab:
    c1[cs][3]=random.randint(1,total_days)
    c1[cs][4]=random.randint(1,time_slots-1)
    c1[cs][5]=random.randint(1,rooms)
    c1[cs][6]=c1[cs][3]
    c1[cs][7]=c1[cs][4]+1
    c1[cs][8]=c1[cs][5]
  else:
    c1[cs][3]=random.randint(1,total_days)
    c1[cs][4]=random.randint(1,time_slots)
    c1[cs][5]=random.randint(1,rooms)
    c1[cs][6]=random.randint(1,total_days)
    c1[cs][7]=random.randint(1,time_slots)
    c1[cs][8]=random.randint(1,rooms)
  return

In [76]:
def mutate_advance(c1,rooms,conflict_info):
  index=conflict_info[0]
  total_days=5
  time_slots=6
  cs=index
  isLab=True if c1[cs][0]==1 else False
  if isLab:
    c1[cs][3]=random.randint(1,total_days)
    c1[cs][4]=random.randint(1,time_slots-1)
    c1[cs][5]=random.randint(1,rooms)
    c1[cs][6]=c1[cs][3]
    c1[cs][7]=c1[cs][4]+1
    c1[cs][8]=c1[cs][5]
  else:
    if conflict_info[1]==1:
      c1[cs][3]=random.randint(1,total_days)
      c1[cs][4]=random.randint(1,time_slots)
      c1[cs][5]=random.randint(1,rooms)
    else:
      c1[cs][6]=random.randint(1,total_days)
      c1[cs][7]=random.randint(1,time_slots)
      c1[cs][8]=random.randint(1,rooms)
  return c1

In [87]:
def evaluate_population(sections,course_sections,profs,classrooms,population,check_soft=False):
  soft_factor=0.001
  conflicts=np.zeros(len(population),dtype=float)
  conflict_info=np.zeros((len(population),2),dtype=int)
  for c in range(len(population)):
    i=population[c]
    room_conflicts=set()
    prof_conflicts=set()
    section_conflicts=set()

    professor_teaching_days={}
    professor_traverse_days={}
    section_days={}

    for j in range(len(i)):
      tokenized_cs=re.split(',',course_sections[1][j])
      section_id=tokenized_cs[0]+tokenized_cs[1]

      #TS1 checks
      section=section_id+','+str(i[j][3])+','+str(i[j][4])
      room=str(i[j][5])+','+str(i[j][3])+','+str(i[j][4])
      prof=str(i[j][2])+','+str(i[j][3])+','+str(i[j][4])
      if room not in room_conflicts:
        room_conflicts.add(room)
      else:
        conflict_info[c][0]=j
        conflict_info[c][1]=1
        conflicts[c]+=1
      if prof not in prof_conflicts:
        prof_conflicts.add(prof)
      else:
        conflict_info[c][0]=j
        conflict_info[c][1]=1
        conflicts[c]+=1
      if section not in section_conflicts:
        section_conflicts.add(section)
      else:
        conflict_info[c][0]=j
        conflict_info[c][1]=1
        conflicts[c]+=1

      #TS2 Checks
      section=section_id+','+str(i[j][6])+','+str(i[j][7])
      room=str(i[j][8])+','+str(i[j][6])+','+str(i[j][7])
      prof=str(i[j][2])+','+str(i[j][6])+','+str(i[j][7])
      if room not in room_conflicts:
        room_conflicts.add(room)
      else:
        conflict_info[c][0]=j
        conflict_info[c][1]=2
        conflicts[c]+=1
      if prof not in prof_conflicts:
        prof_conflicts.add(prof)
      else:
        conflict_info[c][0]=j
        conflict_info[c][1]=2
        conflicts[c]+=1
      if section not in section_conflicts:
        section_conflicts.add(section)
      else:
        conflict_info[c][0]=j
        conflict_info[c][1]=2
        conflicts[c]+=1

      #check if room sizes are good

      room1=classrooms[1][i[j][5]-1]
      room2=classrooms[1][i[j][8]-1]

      token1=re.split(',',room1)
      token2=re.split(',',room2)

      if int(token1[-1])<int(i[j][1]):
        conflict_info[c][0]=j
        conflict_info[c][1]=1
        conflicts[c]+=1
      if int(token2[-1])<int(i[j][1]):
        conflict_info[c][0]=j
        conflict_info[c][1]=2
        conflicts[c]+=1


      #check if lectures on same or adjacent days
      #code this into the chromosome generation/crossover/mutation seems no need
      if abs(i[j][3]-i[j][6])<2 and i[j][0]!=1:
        conflict_info[c][0]=j
        conflict_info[c][1]=random.randint(1,2)
        conflicts[c]+=1

      #soft constraints
      if check_soft==True:

        #checking for same classroom in whole week
        if i[j][5]!=i[j][8]:
          conflict_info[c][0]=j
          conflict_info[c][1]=random.randint(1,2)
          conflicts[c]+=soft_factor

        #checking morning and afternoon sessions
        if i[j][0]==1:
          if i[j][4]<5:
            conflict_info[c][0]=j
            conflict_info[c][1]=random.randint(1,2)
            conflicts[c]+=soft_factor
        else:
          if i[j][7]>4:
            conflict_info[c][0]=j
            conflict_info[c][1]=random.randint(1,2)
            conflicts[c]+=soft_factor
          if i[j][4]>4:
            conflicts[c]+=soft_factor

        #check for floor and block gap bw classes
        #manhattan distance between block/floors of different section,day pairs times soft factor

        #check for uninterrupted teaching
        #absolute distance between professor,day combos times soft factor

        section_traverse_key=section_id+','+str(i[j][3])
        professor_traverse_key=str(i[j][2])+','+str(i[j][3])

        if section_traverse_key not in section_days.keys():
          section_days[section_traverse_key]=[]
        if professor_traverse_key not in professor_traverse_days.keys():
          professor_traverse_days[professor_traverse_key]=[]
          professor_teaching_days[professor_traverse_key]=[]

        section_days[section_traverse_key]+=[[room1,i[j][4]]]
        professor_traverse_days[professor_traverse_key]+=[[room1,i[j][4]]]
        professor_teaching_days[professor_traverse_key]+=[[course_sections[1][j],i[j][4]]]

        section_traverse_key=section_id+','+str(i[j][6])
        professor_traverse_key=str(i[j][2])+','+str(i[j][6])

        if section_traverse_key not in section_days.keys():
          section_days[section_traverse_key]=[]
        if professor_traverse_key not in professor_traverse_days.keys():
          professor_teaching_days[professor_traverse_key]=[]
          professor_traverse_days[professor_traverse_key]=[]

        section_days[section_traverse_key]+=[[room2,i[j][7]]]
        professor_traverse_days[professor_traverse_key]+=[[room2,i[j][7]]]
        professor_teaching_days[professor_traverse_key]+=[[course_sections[1][j],i[j][7]]]
      #loop end
      if check_soft==True:
        x_coords=np.zeros(6)
        y_coords=np.zeros(6)
        manhattan_distance=0
        for i1 in section_days.keys():
          for j1 in section_days[i1]:
            room=re.split(',',j1[0])
            x_coords[j1[1]-1]=ord(room[0])-ord('A')
            y_coords[j1[1]-1]=int(room[1][0])
        for i1 in(range(5)):
          if x_coords[i1]==0 or x_coords[i1+1]:
            continue
          manhattan_distance+=abs(x_coords[i1]-x_coords[i1+1])+abs(y_coords[i1]-y_coords[i1+1])


        x_coords=np.zeros(6)
        y_coords=np.zeros(6)
        for i2 in professor_traverse_days.keys():
          for j2 in professor_traverse_days[i2]:
            room=re.split(',',j2[0])
            x_coords[j2[1]-1]=ord(room[0])-ord('A')
            y_coords[j2[1]-1]=int(room[1][0])
        for i2 in(range(5)):
          if x_coords[i2]==0 or x_coords[i2+1]:
            continue
          manhattan_distance+=abs(x_coords[i2]-x_coords[i2+1])+abs(y_coords[i2]-y_coords[i2+1])
        conflict_info[c][0]=random.randint(0,len(i)-1)
        conflict_info[c][1]=random.randint(1,2)
        conflicts[c]-=manhattan_distance*(soft_factor/5)

        min_idx=-1
        max_idx=6
        isTeaching=np.zeros(6,bool)
        for i3 in professor_traverse_days.keys():
          for j3 in professor_teaching_days[i3]:
            course=re.split(',',j3[0])
            isTeaching[j3[1]-1]=True
            if j3[1]-1<min_idx:
              min_idx=j3[1]
            if j3[1]+1>max_idx:
              max_idx=j3[1]

        for i4 in range(min_idx,max_idx):
          if not isTeaching[i4]:
            conflict_info[c][0]=random.randint(0,len(i)-1)
            conflict_info[c][1]=random.randint(1,2)
            conflicts[c]+=soft_factor/5


    conflicts[c]=-conflicts[c]
  return conflicts,conflict_info

In [78]:
def generate_chromosomes(prof_map,cs_map,rooms):
  #mychromosome=(isLab,section_strength,professor,day1,time1,room1,day2,time2,room2)
  total_days=5
  time_slots=6
  chromosome=np.zeros((len(prof_map),9),dtype=np.int16)

  for i in prof_map.keys():
    tokenized_course_sections=re.split(',',i)
    cs=cs_map[i]
    isLab=len(tokenized_course_sections)==5
    chromosome[cs][1]=int(tokenized_course_sections[2])
    chromosome[cs][2]=prof_map[i]
    if isLab:
      chromosome[cs][0]=1
      chromosome[cs][3]=random.randint(1,total_days)
      chromosome[cs][4]=random.randint(1,time_slots-1)
      chromosome[cs][5]=random.randint(1,len(rooms))
      chromosome[cs][6]=chromosome[cs][3]
      chromosome[cs][7]=chromosome[cs][4]+1
      chromosome[cs][8]=chromosome[cs][5]
    else:
      chromosome[cs][0]=0
      chromosome[cs][3]=random.randint(1,total_days)
      chromosome[cs][4]=random.randint(1,time_slots)
      chromosome[cs][5]=random.randint(1,len(rooms))
      chromosome[cs][6]=random.randint(1,total_days)
      chromosome[cs][7]=random.randint(1,time_slots)
      chromosome[cs][8]=random.randint(1,len(rooms))
  return chromosome

In [79]:
def run_genetic_algo(sections,course_sections,profs,classrooms,check_soft=False,n_pop=6,r_mut=0.5,r_cross=0.6,n_iters=40000,soft_cap=5000):
  course_mapping=index_map(course_sections)
  room_mapping=index_map(classrooms)
  pop_list=list()
  zero_found=False
  for i in range(n_pop):
    pop_list.append(generate_chromosomes(profs[2],course_mapping[0],classrooms))
  population=np.array(pop_list)

  final_iter=n_iters

  soft_iters=0

  memo_scores=np.zeros(n_iters)
  for i in range(n_iters):
    scores,c_info=evaluate_population(sections,course_mapping,profs,room_mapping,population,zero_found and check_soft)
    p=scores.argsort()[::-1]
    population=population[p]
    scores=scores[p]
    c_info=c_info[p]
    best=population[0].copy()
    second=population[1].copy()

    memo_scores[i]=scores[0]
      
    for j in range(0,n_pop-2,2):
      if random.uniform(0,1)<r_cross:
        crossover(population[j],population[j+1])
      if random.uniform(0,1)<r_mut:
        mutate_advance(population[j],len(classrooms),c_info[j])
      if random.uniform(0,1)<r_mut:
        mutate_advance(population[j+1],len(classrooms),c_info[j+1])

    population[-1]=best
    population[-2]=second
    if(scores[0]==0 and not zero_found):
      if(check_soft==False):
        final_iter=i
        return population,scores,final_iter,soft_iters,memo_scores
      zero_found=True
    if zero_found==True:
      soft_iters+=1
      if soft_iters>=soft_cap:
        final_iter=i
        return population,scores,final_iter,soft_iters,memo_scores
    if i%50==0:
        print("Iteration: ", i ," Scores: ", scores)

  return population,scores,final_iter,soft_iters,memo_scores

In [80]:
#Generating data
courses=[['PF','Calculus','Physics','ECCL'],
         ['OOP','DLD','Differential Equations','CPS'],
         ['Data structures','COAL','Discrete','Linear Algebra'],
         ['DB','OS','Algo','Stats'],
         ['C-Net','SE','Automata','Statisical Modelling'],
         ['AI','DIP','Numerical Computing','PDC'],
         ['Info Sec','PPIT'],
         ['DevOps','Computer Vision']]
labs=['PF,Lab','OOP,Lab','DS,Lab','DB,Lab','CNet,Lab','AI,Lab']
sections=generate_sections(1,4,False)
course_sections=generate_course_sections(courses,labs,sections)
profs=assign_profs(courses,labs,sections,course_sections,math.ceil(len(course_sections)/3))
classrooms=generate_classrooms(1,1,20)

In [81]:
print(len(course_sections))

10


In [88]:
#run the genetic algo
total_time=0
n_runs=1
solution=None
sol1=None
for i in range(n_runs):
  start_time = time.time()
  solution=run_genetic_algo(sections,course_sections,profs,classrooms,True,4,1,0.1,60000,3000)
  total_time+=(time.time() - start_time)
  print("Time taken for solution: ", time.time() - start_time, " seconds.")
  len_list=(2,120,math.ceil(len(profs[2])/3),5,6,len(classrooms),5,6,len(classrooms))
  sol1=binary_encode(solution[0][0],len_list)
  
    
average=total_time/n_runs
print("Average runtime is: ",average, " seconds.")

Iteration:  0  Scores:  [-12. -16. -16. -21.]
Iteration:  50  Scores:  [-0.0152 -0.0152 -2.0148 -2.0152]
Iteration:  100  Scores:  [-0.0112 -0.0112 -1.0122 -2.0122]
Iteration:  150  Scores:  [-0.0078 -0.0078 -1.0078 -2.0088]
Iteration:  200  Scores:  [-0.0066 -0.0074 -1.0092 -2.0076]
Iteration:  250  Scores:  [-0.0064 -0.0064 -0.0064 -2.0076]
Iteration:  300  Scores:  [-0.0064 -0.0064 -2.0074 -2.0074]
Iteration:  350  Scores:  [-0.0054 -0.0064 -3.0076 -3.0082]
Iteration:  400  Scores:  [-0.0054 -0.0054 -0.0064 -1.0074]
Iteration:  450  Scores:  [-0.0054 -0.0054 -1.0076 -3.0072]
Iteration:  500  Scores:  [-0.0054 -0.0054 -1.0068 -2.0054]
Iteration:  550  Scores:  [-0.0054 -0.0054 -0.0064 -2.0054]
Iteration:  600  Scores:  [-0.0044 -0.0054 -1.0058 -3.0064]
Iteration:  650  Scores:  [-0.0044 -0.0044 -0.0066 -1.0054]
Iteration:  700  Scores:  [-0.0044 -0.0044 -2.0044 -2.0064]
Iteration:  750  Scores:  [-0.0044 -0.0044 -0.0054 -1.0054]
Iteration:  800  Scores:  [-0.0042 -0.0044 -2.0052 -2.0

In [85]:
write_to_file(solution[0][0],classrooms,course_sections,profs[0])

In [89]:
sol1

[['00011100010000001011001000101000011010001'],
 ['00011100110000011001001001100010001010011'],
 ['00011100010000101000100110000100100001100'],
 ['01011100010010011010101000100110110010001'],
 ['00011100110010001010001000101010001010001'],
 ['01011100110010100010100010101000110000101'],
 ['00011100110100011000101000100010011010001'],
 ['00011100110100100001100110000010010001100'],
 ['00011100010100101001100011000010001000110'],
 ['00011100010110100001000010000100001000100']]

In [ ]:
x=range(0,len(solution[-1][:solution[2]-solution[3]]))
y=solution[-1][:solution[2]-solution[3]]
plt.plot(x,y)
plt.gca().invert_yaxis()
plt.xlabel("Generation number")
plt.ylabel("Fitness")